In [1]:
#Mount the Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Handwritten Digits GAN - Genrative Adversarial Network


## Summary
- Problem Statement
- Import the required libraries - tensorflow
- Pre-process the dataset
- Generator
- Discriminator
- Combining Generator and Discriminator
- Training the Model & Output Visualization

## Problem Statement
- MNIST  - Modified National Institute of Standards and Technology dataset.
- 60,000 training images and 10,000 testing images.
- Each image is 28x28 pixels , a total of 784 pixels.
- Pixel-value is an integer between 0 and 255, inclusive.
- Goal is to correctly differenciate  digits from a dataset of thousands of handwritten images and fake generated image.

![alt text](https://i.imgur.com/Su00XUA.png)

## Import the required libraries - tensorflow

In [2]:
# for data analysis
import pandas as pd
import numpy as np

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # for progress bar

# for model training
import tensorflow as tf # Importing Tensorflow Library
from tensorflow.keras.datasets import mnist # Importing MNIST Dataset
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Activation,Input
from tensorflow.keras.optimizers import Adam


## Pre-process the dataset

In [3]:
class PreProcess:
  def __init__(self):
    self.X_train = None
    self.X_test = None
    self.y_train = None
    self.y_test = None

  def load_data(self):
    try :
      # Loading MNIST dataset
      (self.X_train, self.y_train), (self.X_test, self.y_test) = mnist.load_data()
      # Data Exploration
      print(f"Train data shape :{self.X_train.shape}")
      print(f"Test data shape :{self.X_test.shape}")
    except Exception as e:
      print(f"Failed to load MNIST data : {str(e)}")

  def  reshape_scale(self):
    try :
      #convert data to floating pint and scale to -1 to 1
      # data range - 0 to 255 , (0-127.5)/127.5 = -1 ,(255-127.5)/127.5 = +1
      # we can select scaling parameter any . -1 to +1 is use to tanh activation function instead sigmoid
        self.X_train =(self.X_train.astype('float32') -127.5)/127.5
        self.X_test =(self.X_test.astype('float32') -127.5)/127.5

      #now datasize is 60000 * 28*28 or 10000*28*28
      #reshaping -  we need dataset in 784 pixel.
        self.X_train = self.X_train.reshape(60000, 784)
        self.X_test = self.X_test.reshape(10000, 784)
        print(f"Train data shape after reshape  :{self.X_train.shape}")
        print(f"Test data shape after reshape :{self.X_test.shape}")
        return self.X_train, self.X_test, self.y_train, self.y_test
    except ValueError as ve:
        raise ValueError(f"Reshaping or scaling failed :{str(ve)}")
    except AttributeError as ae:
       raise AttributeError(f"Data not loaded. Call load_data() first: {str(ae)}")
    except Exception as e:
        raise RuntimeError (f"An unexpected error occurred during reshape_scale : {str(e)}")


try:
  pre_process= PreProcess()
  pre_process.load_data()
  X_train, X_test, y_train, y_test = pre_process.reshape_scale()
  print(X_train[0])
except Exception as e:
  print(f"An error occured : {str(e)}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train data shape :(60000, 28, 28)
Test data shape :(10000, 28, 28)
Train data shape after reshape  :(60000, 784)
Test data shape after reshape :(10000, 784)
[-1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.        

#  GAN - Genrative Adversarial Network - Vanilla GAN Model
![alt text](https://i.imgur.com/Fg891TJ.png)

## Generator

In [4]:
class Generaor:
  def __init__(self):
    self.model = None
  def build_generator(self):
    try :
      self.model = Sequential()
    # first layer
      self.model.add(Dense(256,input_dim = 100)) # 100 random samples
      self.model.add(Activation('relu'))
    # second layer
      self.model.add(Dense(512))
      self.model.add(Activation('relu'))
    # third layer
      self.model.add(Dense(1024))
      self.model.add(Activation('relu'))
      #output layer
      self.model.add(Dense(784))
      self.model.add(Activation('tanh')) # as all input has converted -1 to 1 for that reason chosen tan hyperbolic acivation function in last layer.
      # model compilation - 100-256-512-1024-784
      self.model.compile(loss='binary_crossentropy', optimizer='adam')
      return self.model
    except Exception as e:
      print(f"An error occured : {str(e)}")

In [5]:
# Generaor Model
try :
  generator = Generaor()
  gen_model = generator.build_generator()
  gen_model.summary()
except Exception as e:
  print(f"An error occured : {str(e)}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 784)            │       803,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 784)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,486,352 (5.67 MB)

 Trainable params: 1,486,352 (5.67 MB)

 Non-trainable params: 0 (0.00 B)

## Discriminator

In [6]:
class Discriminator:
  def __init__(self):
    self.model = None
  def build_discriminator(self):
    try :
      self.model = Sequential()
    # first layer
      self.model.add(Dense(units=1024,input_dim=784)) # 784 vector  images
      self.model.add(Activation('relu'))
    # second layer
      self.model.add(Dense(512))
      self.model.add(Activation('relu'))
    # third layer
      self.model.add(Dense(256))
      self.model.add(Activation('relu'))
      #output layer
      self.model.add(Dense(1))
      self.model.add(Activation('sigmoid')) # as all input has converted -1 to 1 for that reason chosen tan hyperbolic acivation function in last layer.
      # model compilation - 784-1024-512-256-1 [0-fakes , 1 means real]
      self.model.compile(loss='binary_crossentropy', optimizer='adam')
      return self.model
    except Exception as e:
      print(f"An error occured : {str(e)}")

In [7]:
# Generaor Model
try :
  discriminator = Discriminator()
  des_model = discriminator.build_discriminator()
  des_model.summary()
except Exception as e:
  print(f"An error occured : {str(e)}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1024)           │       803,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,460,225 (5.57 MB)

 Trainable params: 1,460,225 (5.57 MB)

 Non-trainable params: 0 (0.00 B)

## Combining Generator and Discriminator

In [8]:
class GAN:
  def __init__(self, generator, discriminator):
    self.generator = generator
    self.discriminator = discriminator
  def build_gan(self):
    try :
      self.discriminator.trainable = False #don't train the descriminator
      gan_input = Input(shape=(100,)) # 100 dimension vector
      x = self.generator(gan_input) # 100-256-512-1024-784
      gan_output = self.discriminator(x) #784-1024-512-256-1
      gan= Model(inputs=gan_input, outputs=gan_output)  # 100-256-512-1024-784-1024-512-256-1
      gan.compile(loss='binary_crossentropy', optimizer='adam')
      return gan
    except Exception as e:
      print(f"An error occured : {str(e)}")

In [9]:
try:
  gan = GAN(gen_model,des_model)
  gan_model= gan.build_gan()
  gan_model.summary()
except Exception as e:
  print(f"An error occured : {str(e)}")

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 784)            │     1,486,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     1,460,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,946,577 (11.24 MB)

 Trainable params: 1,486,352 (5.67 MB)

 Non-trainable params: 1,460,225 (5.57 MB)

## Training the Model & Output Visualization

In [10]:
#Plot_generated_images to plot the generated images
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    #plt.savefig('gan_generated_image %d.png' %epoch)

In [11]:
class Train:
  def __init__(self,X_train,y_train,X_test,y_test,generator,discriminator,gan):

    #Loading the data
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test

    # Creating GAN
    self.generator = generator # 100-256-512-1024-784
    self.discriminator = discriminator #784-1024-512-256-1
    self.gan = gan  # 100-256-512-1024-784-1024-512-256-1

  def train(self, epochs=50, batch_size=128):
    try :
      batch_count = int(self.X_train.shape[0] / batch_size)
      for epoch in range(epochs):
        for _  in tqdm(range(batch_count)): #tqdm for show progress bar
          #noise vector
          noise = np.random.normal(0, 1, size=(batch_size,100)) #128,100

          #fake images
          fake_images = self.generator.predict(noise)

          #real images
          image_batch = self.X_train[np.random.randint(low=0,high=self.X_train.shape[0],size=batch_size)]  #picking random 128 images from input dataset

          #Construct different batches of  real and fake data
          X= np.concatenate([image_batch, fake_images])

          # Labels for generated and real data
          y_dis=np.zeros(2*batch_size) #128 real images , 128 fake images
          y_dis[:batch_size]=0.9

          #Pre train discriminator on  fake and real data  before starting the gan.
          self.discriminator.trainable=True
          self.discriminator.train_on_batch(X, y_dis)

          #Tricking the noised input of the Generator as real data
          noise= np.random.normal(0,1, [batch_size, 100])
          y_gen = np.ones(batch_size)

          # During the training of gan,
          # the weights of discriminator should be fixed.
          #We can enforce that by setting the trainable flag
          self.discriminator.trainable=False

          #training  the GAN by alternating the training of the Discriminator
          #and training the chained GAN model with Discriminator’s weights freezed.
          self.gan.train_on_batch(noise, y_gen)

        if epoch == 49 or epoch == 50:     # generated images for last 2 epochs
          plot_generated_images(epoch, self.generator)

    except Exception as e:
      print(f"An error occured : {str(e)}")

In [12]:
# commented this code ... as this fails due to require large RAM size.
# try :
#   training = Train(X_train,y_train,X_test,y_test,gen_model,des_model,gan_model)
#   training.train(epochs=50, batch_size=128)
# except Exception as e:
#   print(f"An error occured : {str(e)}")